# Wide ResNet
From: https://github.com/meliketoy/wide-resnet.pytorch/blob/master/networks/wide_resnet.py

Just altered the training script to take different save dir
The requirements.txt is incorrect, will just havr to guess

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, densenet121
from torchvision import datasets, models, transforms


%reload_ext autoreload
%autoreload 2
%matplotlib inline

PATH = Path('/media/rene/data')

# Add the src directory for functions
src_dir = Path.cwd().parent.parent / 'src'
print(src_dir)
sys.path.append(str(src_dir))
sys.path.append('/media/rene/code/wide-resnet.pytorch')


# import my functions:
from utils import make_cfiar10
from functions import*
from models import*
from networks import Wide_ResNet


device="cuda:0"
print(torch.cuda.device_count())
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/code/idc/src
2
True
0


## Find 4 best models

* Is there anything better about using their standard dataloaders rather than the way I did it?

In [9]:
# add directory for the wide resnet
sys.path.append('/media/rene/code/wide-resnet.pytorch')
from networks import *

save_path = '/media/rene/data/cifar-10-batches-py/wide-RN-models'
PATH = '/media/rene/data'
num_workers = 4
batch_size=32
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                   valid_name='valid')

criterion = nn.CrossEntropyLoss()
results = {}

for i in range(6):
    model_name = 'wideRN'+str(i)+'.t7'
    model = Wide_ResNet(28, 20, 0, 10)
    device="cuda:1"
    model = model.to(device)
    
    # load the saved weights
    model_path = os.path.join(save_path, model_name)
    print(model_path)
    model.load_state_dict(torch.load(model_path)['net'].state_dict())

    print('Testing model : ', i)
    valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
    results[model_name] = [valid_acc]
    
f = open(os.path.join(save_path, "wideRN_al68.pkl"),"wb")
pickle.dump(results,f)
f.close()

Files already downloaded and verified
Files already downloaded and verified
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN0.t7
Testing model :  0
Loss: 0.0046 Acc: 0.9601
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN1.t7
Testing model :  1
Loss: 0.0046 Acc: 0.9600
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN2.t7
Testing model :  2
Loss: 0.0046 Acc: 0.9614
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN3.t7
Testing model :  3
Loss: 0.0048 Acc: 0.9600
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN4.t7
Testing model :  4
Loss: 0.0046 Acc: 0.9600
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN5.t7
Testing model :  5
Loss: 0.0045 Acc: 0.9612


In [10]:
import operator

results2 = {key:value[0] for (key,value) in results.items()}
sorted_x = sorted(results2.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_x)

[('wideRN2.t7', 0.9614), ('wideRN5.t7', 0.9612), ('wideRN0.t7', 0.9601), ('wideRN1.t7', 0.96), ('wideRN3.t7', 0.96), ('wideRN4.t7', 0.96)]


# Fusion Model

In [2]:
save_path =  Path(PATH / 'cifar-10-batches-py' / 'wide-RN-models')

model_name_list = ['wideRN2.t7', 'wideRN5.t7', 'wideRN0.t7', 'wideRN1.t7']
num_workers = 6
batch_size = 8
criterion = nn.CrossEntropyLoss()

dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                   valid_name='valid')

pretrained_model_list = []
for i, model_name in enumerate(model_name_list):
    model = Wide_ResNet(28, 20, 0, 10)
    model = model.to(device)
    
    # load the saved weights
    model_path = os.path.join(save_path, model_name)
    print(model_path)
    model.load_state_dict(torch.load(model_path)['net'].state_dict())
    pretrained_model_list.append(model)
    
model = Fusion2(pretrained_model_list, num_input=40, num_output=10)
model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_WRN_1')))
model = model.to(device)

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
print('Fusion2_WRN_1: ', valid_loss, valid_acc)

model = Fusion2(pretrained_model_list, num_input=40, num_output=10)
model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_WRN_2')))
model = model.to(device)

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Fusion2_WRN_2: ', valid_loss, valid_acc)

Files already downloaded and verified
Files already downloaded and verified
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN2.t7
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN5.t7
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN0.t7
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN1.t7
Loss: 0.0227 Acc: 0.9687
Fusion2_WRN_1:  0.02274664867669344 0.9687
Loss: 0.0180 Acc: 0.9672
Fusion2_WRN_2:  0.017988840979337693 0.9672
